# Integrate biogeographic traits with a geographic map

To enable microbiome data analysis in conjunction with the metadata extracted from a geographic map, we also implemented several functions to merge microbial traits with a `SpatialPolygonsDataFrame` or extract a new metadata from a `SpatialPolygonsDataFrame`. By using these functions, we can perform the statistical comparing based on administrative areas or grids. 

Here we need three R packages for this section of [microgeo](https://github.com/ChaonanLi/microgeo) R package tutorial. Just run the following codes to import them into R environment.

In [1]:
suppressMessages(require("magrittr")) 
require("ggplot2")  %>% suppressMessages()
require("microgeo") %>% suppressMessages()

If the Chinese characters cannot be displayed correctly, please run the following codes to set locale to `UTF-8`:

In [2]:
prev_locale <- Sys.setlocale("LC_CTYPE", "C.UTF-8") 

We need a standard microgeo dataset for the presentations in the section of tutorial.

In [3]:
# Use the map downloaded from DataV.GeoAtlas
data(qtp)
map <- read_aliyun_map(adcode = c(540000, 630000, 510000)) %>% suppressMessages() 
dataset.dts.aliyun <- create_dataset(mat = qtp$asv, ant = qtp$tax, met = qtp$met, map = map,
                                     phy = qtp$tre, env = qtp$env, lon = "longitude", lat = "latitude")
dataset.dts.aliyun %<>% rarefy_count_table()
dataset.dts.aliyun %<>% tidy_dataset()
dataset.dts.aliyun %<>% calc_alpha_div(measures = c("observed", "shannon")) 
dataset.dts.aliyun %<>% calc_beta_div(measures = c("bray", "jaccard")) 
dataset.dts.aliyun %>% show_dataset()

ℹ [2024-06-09 19:22:10] INFO ==> all samples fall within the map area!

ℹ [2024-06-09 19:22:10] INFO ==> dataset has been created successfully!

ℹ [2024-06-09 19:22:10] INFO ==> use `object %>% show_dataset()` to check the summary of dataset.

ℹ [2024-06-09 19:22:13] INFO ==> the ASV/gene abundance table has been rarefied with a sub-sample depth of 5310

✔ [2024-06-09 19:22:17] SAVE ==> results have been saved to: object$div$alpha

✔ [2024-06-09 19:23:00] SAVE ==> results have been saved to: object$div$beta



── The Summary of Microgeo Dataset ─────────────────────────────────────────────


ℹ object$mat: 6808 ASVs/genes and 1244 samples [subsample depth: 5310]

ℹ object$ant: 6808 ASVs/genes and 7 annotation levels (Kingdom, Phylum, Class, Order, Family, Genus, Species)

ℹ object$met: 1244 samples and 2 variables (longitude, latitude)

ℹ object$map: a SpatialPolygonsDataFrame with the CRS of '+proj=longlat +datum=WGS84 +no_defs'

ℹ object$phy: a phylogenetic tree with 6808 tip labels

ℹ object$env: 1244 samples and 10 variables




── The Summary of Biogeographic Traits ─────────────────────────────────────────


✔ object$div$alpha: 2 alpha diversity index/indices (observed, shannon)

✔ object$div$beta: 2 beta diversity distance matrix/matrices (bray, jaccard)




• To check the summary of dataset, Replace `object` with the variable name of your dataset
• For example, if the variable name is `dataset.dts`you can run `head(dataset.dts$met)` to check the content of `met`


Now, let's go through each of these functions and see how they are used.

## 1. Merge a `data.frame` with a map

Firstly, we check the `data.frame` of alpha diversity indices, and the `SpatialPolygonsDataFrame`.

In [4]:
# Check the data.frame of alpha diversity indices 
head(dataset.dts.aliyun$div$alpha)

,observed,shannon
,<dbl>,<dbl>
s1,997,6.317993
s2,931,6.221955
s3,833,6.128113
s4,1039,6.258115
s5,882,6.192494
s6,999,6.231846


In [5]:
# Check the `SpatialPolygonsDataFrame`
head(dataset.dts.aliyun$map@data)

,TYPE,FMTS,NAME,X.CENTER,Y.CENTER
,<chr>,<chr>,<chr>,<dbl>,<dbl>
1,DataV.GeoAtlas,microgeo,西藏自治区,88.38828,31.56375
2,DataV.GeoAtlas,microgeo,青海省,96.04353,35.72640
3,DataV.GeoAtlas,microgeo,四川省,102.69345,30.67454


In [6]:
# Change the names of Polygons
dataset.dts.aliyun$map@data$NAME <- c("Tibet", "Qinghai", "Sichuan") 
head(dataset.dts.aliyun$map@data)

,TYPE,FMTS,NAME,X.CENTER,Y.CENTER
,<chr>,<chr>,<chr>,<dbl>,<dbl>
1,DataV.GeoAtlas,microgeo,Tibet,88.38828,31.56375
2,DataV.GeoAtlas,microgeo,Qinghai,96.04353,35.72640
3,DataV.GeoAtlas,microgeo,Sichuan,102.69345,30.67454


Then, we merge the `data.frame` of alpha diversity indices with a `SpatialPolygonsDataFrame`.

In [7]:
# Merge data to a `SpatialPolygonsDataFrame`
common.map.mean4df <- merge_dfs_to_map(map = dataset.dts.aliyun$map, dat = dataset.dts.aliyun$div$alpha, 
                                       met = dataset.dts.aliyun$met, med = 'mean')
head(common.map.mean4df@data[,1:12])
# Now, you can visualize the microbial traits (alpha diversity indices) onto a map

,TYPE,FMTS,NAME,X.CENTER,Y.CENTER,observed_mean,shannon_mean,observed_sd,shannon_sd,observed_se,shannon_se,sample.num
,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<int>
1,DataV.GeoAtlas,microgeo,Tibet,88.38828,31.56375,663.5095,5.847969,238.3167,0.4755891,10.41091,0.02077620,524
2,DataV.GeoAtlas,microgeo,Qinghai,96.04353,35.72640,648.1290,5.837897,246.2957,0.5385612,11.71509,0.02561674,442
3,DataV.GeoAtlas,microgeo,Sichuan,102.69345,30.67454,706.5180,5.973561,197.9724,0.3659978,11.87360,0.02195109,278


We also can merge the `data.frame` of alpha diversity indices with a gridded `SpatialPolygonsDataFrame`. 

In [8]:
# Grid the map [`SpatialPolygonsDataFrame`]
gridded.map <- grid_map(map = dataset.dts.aliyun$map, res = 1.5) %>% suppressMessages
head(gridded.map@data)

,TYPE,FMTS,NAME,X.CENTER,Y.CENTER
,<chr>,<chr>,<chr>,<dbl>,<dbl>
1,Gridded.Map,microgeo,1,83.74702,29.73742
2,Gridded.Map,microgeo,2,85.46302,28.50944
3,Gridded.Map,microgeo,3,86.67299,28.33105
4,Gridded.Map,microgeo,4,89.49169,28.25211
5,Gridded.Map,microgeo,5,88.12468,28.29693
6,Gridded.Map,microgeo,6,85.14919,29.46224


In [9]:
# Merge data to a gridded map
gridded.map.mean4df <- merge_dfs_to_map(map = gridded.map, dat = dataset.dts.aliyun$div$alpha, 
                                        met = dataset.dts.aliyun$met, med = 'mean')
head(gridded.map.mean4df@data[,1:12])
# Now, you can visualize the microbial traits (alpha diversity indices) onto a map

,TYPE,FMTS,NAME,X.CENTER,Y.CENTER,observed_mean,shannon_mean,observed_sd,shannon_sd,observed_se,shannon_se,sample.num
,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<int>
1,Gridded.Map,microgeo,1,83.74702,29.73742,592.7692,5.811758,150.2477,0.2217618,41.67120,0.06150565,13
2,Gridded.Map,microgeo,2,85.46302,28.50944,700.6667,5.950323,203.3822,0.2875940,117.42279,0.16604250,3
3,Gridded.Map,microgeo,3,86.67299,28.33105,554.0000,5.783567,145.5804,0.2778320,40.37675,0.07705673,13
4,Gridded.Map,microgeo,4,89.49169,28.25211,667.5000,5.788181,309.3073,0.5658123,154.65365,0.28290616,4
5,Gridded.Map,microgeo,5,88.12468,28.29693,470.8000,5.424761,152.4140,0.4160093,39.35313,0.10741315,15
6,Gridded.Map,microgeo,6,85.14919,29.46224,554.5417,5.750960,144.9923,0.2275268,29.59644,0.04644371,24


## 2. Merge a `distance matrix` with a map

Firstly, we check the distance `matrix`, and the `SpatialPolygonsDataFrame`.

In [10]:
# Check the distance matrix 
dataset.dts.aliyun$div$beta$bray[1:5, 1:5]

,s1,s2,s3,s4,s5
s1,0.0000000,0.4828625,0.5873823,0.5001883,0.3935970
s2,0.4828625,0.0000000,0.4879473,0.4094162,0.3952919
s3,0.5873823,0.4879473,0.0000000,0.4067797,0.4885122
s4,0.5001883,0.4094162,0.4067797,0.0000000,0.4148776
s5,0.3935970,0.3952919,0.4885122,0.4148776,0.0000000


In [11]:
# Check the `SpatialPolygonsDataFrame`
head(dataset.dts.aliyun$map@data)

,TYPE,FMTS,NAME,X.CENTER,Y.CENTER
,<chr>,<chr>,<chr>,<dbl>,<dbl>
1,DataV.GeoAtlas,microgeo,Tibet,88.38828,31.56375
2,DataV.GeoAtlas,microgeo,Qinghai,96.04353,35.72640
3,DataV.GeoAtlas,microgeo,Sichuan,102.69345,30.67454


Then, we merge the distance `matrix` with a `SpatialPolygonsDataFrame`. 

In [12]:
# Merge distance matrix to a common map
common.map.mean4mx <- merge_mtx_to_map(map = dataset.dts.aliyun$map, dat = dataset.dts.aliyun$div$beta$bray, 
                                        met = dataset.dts.aliyun$met, var = 'bray', med = 'mean')
head(common.map.mean4mx@data[,1:9])
# Now, you can visualize the microbial traits (beta diversity distance matrix) onto a map

,TYPE,FMTS,NAME,X.CENTER,Y.CENTER,bray_mean,bray_sd,bray_se,sample.num
,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<int>
1,DataV.GeoAtlas,microgeo,Tibet,88.38828,31.56375,0.8129844,0.1042697,0.0002816805,524
2,DataV.GeoAtlas,microgeo,Qinghai,96.04353,35.72640,0.7973101,0.1221973,0.0003914227,442
3,DataV.GeoAtlas,microgeo,Sichuan,102.69345,30.67454,0.7214043,0.1389998,0.0007083811,278


We also can merge a distance `matrix` with a gridded `SpatialPolygonsDataFrame`.

In [13]:
# Grid the map 
gridded.map <- grid_map(map = dataset.dts.aliyun$map, res = 1.5) %>% suppressMessages
head(gridded.map@data)

,TYPE,FMTS,NAME,X.CENTER,Y.CENTER
,<chr>,<chr>,<chr>,<dbl>,<dbl>
1,Gridded.Map,microgeo,1,83.74702,29.73742
2,Gridded.Map,microgeo,2,85.46302,28.50944
3,Gridded.Map,microgeo,3,86.67299,28.33105
4,Gridded.Map,microgeo,4,89.49169,28.25211
5,Gridded.Map,microgeo,5,88.12468,28.29693
6,Gridded.Map,microgeo,6,85.14919,29.46224


In [14]:
# Merge distance matrix to a gridded map
gridded.map.mean4mx <- merge_mtx_to_map(map = gridded.map, dat = dataset.dts.aliyun$div$beta$bray, 
                                        met = dataset.dts.aliyun$met, var = 'bray', med = 'mean')
head(gridded.map.mean4mx@data[,1:9])
# Now, you can visualize the microbial traits (beta diversity distance matrix) onto a map

,TYPE,FMTS,NAME,X.CENTER,Y.CENTER,bray_mean,bray_sd,bray_se,sample.num
,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<int>
1,Gridded.Map,microgeo,1,83.74702,29.73742,0.6629630,0.09284562,0.010512697,13
2,Gridded.Map,microgeo,2,85.46302,28.50944,0.6391086,0.09982613,0.057634640,3
3,Gridded.Map,microgeo,3,86.67299,28.33105,0.7319016,0.13026073,0.014749123,13
4,Gridded.Map,microgeo,4,89.49169,28.25211,0.7632454,0.23259699,0.094957324,4
5,Gridded.Map,microgeo,5,88.12468,28.29693,0.7759196,0.13614340,0.013286235,15
6,Gridded.Map,microgeo,6,85.14919,29.46224,0.7701507,0.10856920,0.006535098,24


## 3. Extract metadata table from a map

In [15]:
# Extract metadata from a common map
# Rownames are sample IDs
# This new matadata table can be used for subsequent statistical analysis
metadata <- dataset.dts.aliyun$map %>% extract_metadata_from_map(met = dataset.dts.aliyun$met)
head(metadata)

,longitude,latitude,NAME,TYPE,FMTS,X.CENTER,Y.CENTER
,<dbl>,<dbl>,<chr>,<chr>,<chr>,<dbl>,<dbl>
s1,98.20894,33.10321,Sichuan,DataV.GeoAtlas,microgeo,102.6935,30.67454
s2,98.20894,33.10321,Sichuan,DataV.GeoAtlas,microgeo,102.6935,30.67454
s3,98.20894,33.10321,Sichuan,DataV.GeoAtlas,microgeo,102.6935,30.67454
s4,98.20894,33.10321,Sichuan,DataV.GeoAtlas,microgeo,102.6935,30.67454
s5,98.20894,33.10321,Sichuan,DataV.GeoAtlas,microgeo,102.6935,30.67454
s6,98.20639,33.10280,Sichuan,DataV.GeoAtlas,microgeo,102.6935,30.67454


In [16]:
# Extract metadata from a common map with additional data
# Rownames are sample IDs
# This new matadata table can be used for subsequent statistical analysis 
metadata.from.c.df <- common.map.mean4df %>% extract_metadata_from_map(met = dataset.dts.aliyun$met)
head(metadata.from.c.df)

,longitude,latitude,NAME,TYPE,FMTS,X.CENTER,Y.CENTER,observed_mean,shannon_mean,observed_sd,shannon_sd,observed_se,shannon_se,sample.num
,<dbl>,<dbl>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<int>
s1,98.20894,33.10321,Sichuan,DataV.GeoAtlas,microgeo,102.6935,30.67454,706.518,5.973561,197.9724,0.3659978,11.8736,0.02195109,278
s2,98.20894,33.10321,Sichuan,DataV.GeoAtlas,microgeo,102.6935,30.67454,706.518,5.973561,197.9724,0.3659978,11.8736,0.02195109,278
s3,98.20894,33.10321,Sichuan,DataV.GeoAtlas,microgeo,102.6935,30.67454,706.518,5.973561,197.9724,0.3659978,11.8736,0.02195109,278
s4,98.20894,33.10321,Sichuan,DataV.GeoAtlas,microgeo,102.6935,30.67454,706.518,5.973561,197.9724,0.3659978,11.8736,0.02195109,278
s5,98.20894,33.10321,Sichuan,DataV.GeoAtlas,microgeo,102.6935,30.67454,706.518,5.973561,197.9724,0.3659978,11.8736,0.02195109,278
s6,98.20639,33.10280,Sichuan,DataV.GeoAtlas,microgeo,102.6935,30.67454,706.518,5.973561,197.9724,0.3659978,11.8736,0.02195109,278


In [17]:
# Extract metadata from a gridded map
# Rownames are sample IDs
# This new matadata table can be used for subsequent statistical analysis
metadata.from.g.mx <- gridded.map.mean4mx %>% extract_metadata_from_map(met = dataset.dts.aliyun$met)
head(metadata.from.g.mx)

,longitude,latitude,NAME,TYPE,FMTS,X.CENTER,Y.CENTER,bray_mean,bray_sd,bray_se,sample.num
,<dbl>,<dbl>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<int>
s1,98.20894,33.10321,138,Gridded.Map,microgeo,98.70691,32.50087,0.6562307,0.1576439,0.005010251,45
s2,98.20894,33.10321,138,Gridded.Map,microgeo,98.70691,32.50087,0.6562307,0.1576439,0.005010251,45
s3,98.20894,33.10321,138,Gridded.Map,microgeo,98.70691,32.50087,0.6562307,0.1576439,0.005010251,45
s4,98.20894,33.10321,138,Gridded.Map,microgeo,98.70691,32.50087,0.6562307,0.1576439,0.005010251,45
s5,98.20894,33.10321,138,Gridded.Map,microgeo,98.70691,32.50087,0.6562307,0.1576439,0.005010251,45
s6,98.20639,33.10280,138,Gridded.Map,microgeo,98.70691,32.50087,0.6562307,0.1576439,0.005010251,45
